 모든 데이터를 다 사용해서 LGBM 모델의 파라미터를 튜닝해 보자.

In [1]:
!pip install bayesian-optimization

In [2]:
# 모듈 불러오기
import warnings
warnings.filterwarnings(action='ignore')

import os
os.chdir('/content/drive/My Drive/Big-Contest')

import pandas as pd
pd.set_option('display.max_columns', None)
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import KFold
# from sklearn.decomposition import KernelPCA, PCA

from bayes_opt import BayesianOptimization
import lightgbm as lgb

from functools import partial
import seaborn as sns
import matplotlib.pyplot as plt

In [3]:
# 데이터 로드
data = pd.read_csv("./data/input(raw)_3.csv", usecols=lambda x:'Unnamed' not in x)
data

,AMT,year,month,day,day_of_week,hour,minute,exposure,is_continuous,pcategory,pname,price,exposure_cnt_1,exposure_cnt_2,exposure_cnt_3,exposure_cnt_4,exposure_cnt_5,exposure_cnt_6,exposure_cnt_7,exposure_cnt_8,exposure_cnt_9,exposure_cnt_10,exposure_cnt_11,exposure_cnt_12,exposure_cnt_sum,warning,is_warning,warning_bin4,warning_bin3,alert,is_alert,dust_seoul,dust_busan,dust_daegu,dust_incheon,dust_gwangju,dust_daejeon,dust_ulsan,dust_gyeonggi,dust_gangwon,dust_chungbuk,dust_chungnam,dust_cheonbuk,dust_cheonnam,dust_gyeongbuk,dust_gyeongnam,dust_sejong,daily_temperature_mean,daily_temperature_max,daily_temperature_min,daily_temperature_diff,daily_temperature_mean_1,daily_temperature_max_1,daily_temperature_min_1,daily_temperature_diff_1,daily_temperature_mean_2,daily_temperature_max_2,daily_temperature_min_2,daily_precipitation_sum,daily_humidity_mean,daily_humidity_max,daily_humidity_mean_1,daily_humidity_mean_2,daily_humidity_max_2,daily_sunshine_sum,daily_sunshine_max,daily_sunshine_1,daily_insolation_sum,daily_snow_sum,gonghang_20,pildong_40,gonghang_50,jongro56_30,samcheong_40,jongro56_40,gonghang_40,jegi_40,pildong_30,sogong_20,samcheong_50,sogong_30,gonghang_30,sogong_40,gonghang_20_cat,pildong_40_cat,gonghang_50_cat,jongro56_30_cat,samcheong_40_cat,jongro56_40_cat,gonghang_40_cat,jegi_40_cat,pildong_30_cat,sogong_20_cat,samcheong_50_cat,sogong_30_cat,gonghang_30_cat,sogong_40_cat
0,4371000,2019,0,0,1,3,0,17,0,6,1528,39900,5,0,0,0,0,0,0,0,0,0,0,0,5,0,0,2,2,0,0,25.0,18.0,18.0,23.0,19.0,30.0,16.0,27.0,25.0,34.0,23.0,30.0,15.0,21.0,15.0,30.0,-4.770833,-0.9,-7.9,7.0,7,7,7,2,5,5,5,0.0,50.125000,62.0,9,5,5,7.5,1.0,3,7.84,0.0,0.324824,0.188639,0.338075,0.160344,0.240295,0.159082,0.330859,0.346455,0.216588,0.133736,0.248899,0.110893,0.342265,0.115280,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,2099000,2019,0,0,1,3,0,17,0,6,1522,39900,5,0,0,0,0,0,0,0,0,0,0,0,5,0,0,2,2,0,0,25.0,18.0,18.0,23.0,19.0,30.0,16.0,27.0,25.0,34.0,23.0,30.0,15.0,21.0,15.0,30.0,-4.770833,-0.9,-7.9,7.0,7,7,7,2,5,5,5,0.0,50.125000,62.0,9,5,5,7.5,1.0,3,7.84,0.0,0.324824,0.188639,0.338075,0.160344,0.240295,0.159082,0.330859,0.346455,0.216588,0.133736,0.248899,0.110893,0.342265,0.115280,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,3262000,2019,0,0,1,3,4,17,1,6,1522,39900,5,0,0,0,0,0,0,0,0,0,0,0,5,0,0,2,2,0,0,25.0,18.0,18.0,23.0,19.0,30.0,16.0,27.0,25.0,34.0,23.0,30.0,15.0,21.0,15.0,30.0,-4.770833,-0.9,-7.9,7.0,7,7,7,2,5,5,5,0.0,50.125000,62.0,9,5,5,7.5,1.0,3,7.84,0.0,0.324824,0.188639,0.338075,0.160344,0.240295,0.159082,0.330859,0.346455,0.216588,0.133736,0.248899,0.110893,0.342265,0.115280,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,6955000,2019,0,0,1,3,4,17,1,6,1528,39900,5,0,0,0,0,0,0,0,0,0,0,0,5,0,0,2,2,0,0,25.0,18.0,18.0,23.0,19.0,30.0,16.0,27.0,25.0,34.0,23.0,30.0,15.0,21.0,15.0,30.0,-4.770833,-0.9,-7.9,7.0,7,7,7,2,5,5,5,0.0,50.125000,62.0,9,5,5,7.5,1.0,3,7.84,0.0,0.324824,0.188639,0.338075,0.160344,0.240295,0.159082,0.330859,0.346455,0.216588,0.133736,0.248899,0.110893,0.342265,0.115280,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,6672000,2019,0,0,1,3,7,17,2,6,1522,39900,5,0,0,0,0,0,0,0,0,0,0,0,5,0,0,2,2,0,0,25.0,18.0,18.0,23.0,19.0,30.0,16.0,27.0,25.0,34.0,23.0,30.0,15.0,21.0,15.0,30.0,-4.770833,-0.9,-7.9,7.0,7,7,7,2,5,5,5,0.0,50.125000,62.0,9,5,5,7.5,1.0,3,7.84,0.0,0.324824,0.188639,0.338075,0.160344,0.240295,0.159082,0.330859,0.346455,0.216588,0.133736,0.248899,0.110893,0.342265,0.115280,0,0,0,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37363,4573000,2019,11,11,1,20,4,17,0,9,793,178000,1,0,0,0,0,0,0,0,0,0,7,26,34,0,0,2,2,0,0,15.0,12.0,11.0,12.0,9.0,11.0,10.0,13.0,14.0,15.0,13.0,11.0,8.0,12.0,8.0,11.0,-7.841667,-4.9,-10.8,5.9,7,7,7,2,5,5,5,0.0,41.6

In [4]:
# 취급액 0인 데이터, year 컬럼 제외
df = data.copy()
df = df.loc[df['AMT'] != 0].reset_index(drop=True)
df = df.drop('year', axis=1)
df

,AMT,month,day,day_of_week,hour,minute,exposure,is_continuous,pcategory,pname,price,exposure_cnt_1,exposure_cnt_2,exposure_cnt_3,exposure_cnt_4,exposure_cnt_5,exposure_cnt_6,exposure_cnt_7,exposure_cnt_8,exposure_cnt_9,exposure_cnt_10,exposure_cnt_11,exposure_cnt_12,exposure_cnt_sum,warning,is_warning,warning_bin4,warning_bin3,alert,is_alert,dust_seoul,dust_busan,dust_daegu,dust_incheon,dust_gwangju,dust_daejeon,dust_ulsan,dust_gyeonggi,dust_gangwon,dust_chungbuk,dust_chungnam,dust_cheonbuk,dust_cheonnam,dust_gyeongbuk,dust_gyeongnam,dust_sejong,daily_temperature_mean,daily_temperature_max,daily_temperature_min,daily_temperature_diff,daily_temperature_mean_1,daily_temperature_max_1,daily_temperature_min_1,daily_temperature_diff_1,daily_temperature_mean_2,daily_temperature_max_2,daily_temperature_min_2,daily_precipitation_sum,daily_humidity_mean,daily_humidity_max,daily_humidity_mean_1,daily_humidity_mean_2,daily_humidity_max_2,daily_sunshine_sum,daily_sunshine_max,daily_sunshine_1,daily_insolation_sum,daily_snow_sum,gonghang_20,pildong_40,gonghang_50,jongro56_30,samcheong_40,jongro56_40,gonghang_40,jegi_40,pildong_30,sogong_20,samcheong_50,sogong_30,gonghang_30,sogong_40,gonghang_20_cat,pildong_40_cat,gonghang_50_cat,jongro56_30_cat,samcheong_40_cat,jongro56_40_cat,gonghang_40_cat,jegi_40_cat,pildong_30_cat,sogong_20_cat,samcheong_50_cat,sogong_30_cat,gonghang_30_cat,sogong_40_cat
0,4371000,0,0,1,3,0,17,0,6,1528,39900,5,0,0,0,0,0,0,0,0,0,0,0,5,0,0,2,2,0,0,25.0,18.0,18.0,23.0,19.0,30.0,16.0,27.0,25.0,34.0,23.0,30.0,15.0,21.0,15.0,30.0,-4.770833,-0.9,-7.9,7.0,7,7,7,2,5,5,5,0.0,50.125000,62.0,9,5,5,7.5,1.0,3,7.84,0.0,0.324824,0.188639,0.338075,0.160344,0.240295,0.159082,0.330859,0.346455,0.216588,0.133736,0.248899,0.110893,0.342265,0.115280,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,2099000,0,0,1,3,0,17,0,6,1522,39900,5,0,0,0,0,0,0,0,0,0,0,0,5,0,0,2,2,0,0,25.0,18.0,18.0,23.0,19.0,30.0,16.0,27.0,25.0,34.0,23.0,30.0,15.0,21.0,15.0,30.0,-4.770833,-0.9,-7.9,7.0,7,7,7,2,5,5,5,0.0,50.125000,62.0,9,5,5,7.5,1.0,3,7.84,0.0,0.324824,0.188639,0.338075,0.160344,0.240295,0.159082,0.330859,0.346455,0.216588,0.133736,0.248899,0.110893,0.342265,0.115280,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,3262000,0,0,1,3,4,17,1,6,1522,39900,5,0,0,0,0,0,0,0,0,0,0,0,5,0,0,2,2,0,0,25.0,18.0,18.0,23.0,19.0,30.0,16.0,27.0,25.0,34.0,23.0,30.0,15.0,21.0,15.0,30.0,-4.770833,-0.9,-7.9,7.0,7,7,7,2,5,5,5,0.0,50.125000,62.0,9,5,5,7.5,1.0,3,7.84,0.0,0.324824,0.188639,0.338075,0.160344,0.240295,0.159082,0.330859,0.346455,0.216588,0.133736,0.248899,0.110893,0.342265,0.115280,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,6955000,0,0,1,3,4,17,1,6,1528,39900,5,0,0,0,0,0,0,0,0,0,0,0,5,0,0,2,2,0,0,25.0,18.0,18.0,23.0,19.0,30.0,16.0,27.0,25.0,34.0,23.0,30.0,15.0,21.0,15.0,30.0,-4.770833,-0.9,-7.9,7.0,7,7,7,2,5,5,5,0.0,50.125000,62.0,9,5,5,7.5,1.0,3,7.84,0.0,0.324824,0.188639,0.338075,0.160344,0.240295,0.159082,0.330859,0.346455,0.216588,0.133736,0.248899,0.110893,0.342265,0.115280,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,6672000,0,0,1,3,7,17,2,6,1522,39900,5,0,0,0,0,0,0,0,0,0,0,0,5,0,0,2,2,0,0,25.0,18.0,18.0,23.0,19.0,30.0,16.0,27.0,25.0,34.0,23.0,30.0,15.0,21.0,15.0,30.0,-4.770833,-0.9,-7.9,7.0,7,7,7,2,5,5,5,0.0,50.125000,62.0,9,5,5,7.5,1.0,3,7.84,0.0,0.324824,0.188639,0.338075,0.160344,0.240295,0.159082,0.330859,0.346455,0.216588,0.133736,0.248899,0.110893,0.342265,0.115280,0,0,0,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37363,4573000,11,11,1,20,4,17,0,9,793,178000,1,0,0,0,0,0,0,0,0,0,7,26,34,0,0,2,2,0,0,15.0,12.0,11.0,12.0,9.0,11.0,10.0,13.0,14.0,15.0,13.0,11.0,8.0,12.0,8.0,11.0,-7.841667,-4.9,-10.8,5.9,7,7,7,2,5,5,5,0.0,41.666667,66.0,7,5,5,9.1,1.0,3,11.11,0.0,0.

In [5]:
# input, feature 분리
X_data = df.iloc[:, 1:]
y_data = df[['AMT']]
display(X_data)
display(y_data)

,month,day,day_of_week,hour,minute,exposure,is_continuous,pcategory,pname,price,exposure_cnt_1,exposure_cnt_2,exposure_cnt_3,exposure_cnt_4,exposure_cnt_5,exposure_cnt_6,exposure_cnt_7,exposure_cnt_8,exposure_cnt_9,exposure_cnt_10,exposure_cnt_11,exposure_cnt_12,exposure_cnt_sum,warning,is_warning,warning_bin4,warning_bin3,alert,is_alert,dust_seoul,dust_busan,dust_daegu,dust_incheon,dust_gwangju,dust_daejeon,dust_ulsan,dust_gyeonggi,dust_gangwon,dust_chungbuk,dust_chungnam,dust_cheonbuk,dust_cheonnam,dust_gyeongbuk,dust_gyeongnam,dust_sejong,daily_temperature_mean,daily_temperature_max,daily_temperature_min,daily_temperature_diff,daily_temperature_mean_1,daily_temperature_max_1,daily_temperature_min_1,daily_temperature_diff_1,daily_temperature_mean_2,daily_temperature_max_2,daily_temperature_min_2,daily_precipitation_sum,daily_humidity_mean,daily_humidity_max,daily_humidity_mean_1,daily_humidity_mean_2,daily_humidity_max_2,daily_sunshine_sum,daily_sunshine_max,daily_sunshine_1,daily_insolation_sum,daily_snow_sum,gonghang_20,pildong_40,gonghang_50,jongro56_30,samcheong_40,jongro56_40,gonghang_40,jegi_40,pildong_30,sogong_20,samcheong_50,sogong_30,gonghang_30,sogong_40,gonghang_20_cat,pildong_40_cat,gonghang_50_cat,jongro56_30_cat,samcheong_40_cat,jongro56_40_cat,gonghang_40_cat,jegi_40_cat,pildong_30_cat,sogong_20_cat,samcheong_50_cat,sogong_30_cat,gonghang_30_cat,sogong_40_cat
0,0,0,1,3,0,17,0,6,1528,39900,5,0,0,0,0,0,0,0,0,0,0,0,5,0,0,2,2,0,0,25.0,18.0,18.0,23.0,19.0,30.0,16.0,27.0,25.0,34.0,23.0,30.0,15.0,21.0,15.0,30.0,-4.770833,-0.9,-7.9,7.0,7,7,7,2,5,5,5,0.0,50.125000,62.0,9,5,5,7.5,1.0,3,7.84,0.0,0.324824,0.188639,0.338075,0.160344,0.240295,0.159082,0.330859,0.346455,0.216588,0.133736,0.248899,0.110893,0.342265,0.115280,0,0,0,0,0,0,0,1,0,0,0,0,1,0
1,0,0,1,3,0,17,0,6,1522,39900,5,0,0,0,0,0,0,0,0,0,0,0,5,0,0,2,2,0,0,25.0,18.0,18.0,23.0,19.0,30.0,16.0,27.0,25.0,34.0,23.0,30.0,15.0,21.0,15.0,30.0,-4.770833,-0.9,-7.9,7.0,7,7,7,2,5,5,5,0.0,50.125000,62.0,9,5,5,7.5,1.0,3,7.84,0.0,0.324824,0.188639,0.338075,0.160344,0.240295,0.159082,0.330859,0.346455,0.216588,0.133736,0.248899,0.110893,0.342265,0.115280,0,0,0,0,0,0,0,1,0,0,0,0,1,0
2,0,0,1,3,4,17,1,6,1522,39900,5,0,0,0,0,0,0,0,0,0,0,0,5,0,0,2,2,0,0,25.0,18.0,18.0,23.0,19.0,30.0,16.0,27.0,25.0,34.0,23.0,30.0,15.0,21.0,15.0,30.0,-4.770833,-0.9,-7.9,7.0,7,7,7,2,5,5,5,0.0,50.125000,62.0,9,5,5,7.5,1.0,3,7.84,0.0,0.324824,0.188639,0.338075,0.160344,0.240295,0.159082,0.330859,0.346455,0.216588,0.133736,0.248899,0.110893,0.342265,0.115280,0,0,0,0,0,0,0,1,0,0,0,0,1,0
3,0,0,1,3,4,17,1,6,1528,39900,5,0,0,0,0,0,0,0,0,0,0,0,5,0,0,2,2,0,0,25.0,18.0,18.0,23.0,19.0,30.0,16.0,27.0,25.0,34.0,23.0,30.0,15.0,21.0,15.0,30.0,-4.770833,-0.9,-7.9,7.0,7,7,7,2,5,5,5,0.0,50.125000,62.0,9,5,5,7.5,1.0,3,7.84,0.0,0.324824,0.188639,0.338075,0.160344,0.240295,0.159082,0.330859,0.346455,0.216588,0.133736,0.248899,0.110893,0.342265,0.115280,0,0,0,0,0,0,0,1,0,0,0,0,1,0
4,0,0,1,3,7,17,2,6,1522,39900,5,0,0,0,0,0,0,0,0,0,0,0,5,0,0,2,2,0,0,25.0,18.0,18.0,23.0,19.0,30.0,16.0,27.0,25.0,34.0,23.0,30.0,15.0,21.0,15.0,30.0,-4.770833,-0.9,-7.9,7.0,7,7,7,2,5,5,5,0.0,50.125000,62.0,9,5,5,7.5,1.0,3,7.84,0.0,0.324824,0.188639,0.338075,0.160344,0.240295,0.159082,0.330859,0.346455,0.216588,0.133736,0.248899,0.110893,0.342265,0.115280,0,0,0,0,0,0,0,1,0,0,0,0,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
37363,11,11,1,20,4,17,0,9,793,178000,1,0,0,0,0,0,0,0,0,0,7,26,34,0,0,2,2,0,0,15.0,12.0,11.0,12.0,9.0,11.0,10.0,13.0,14.0,15.0,13.0,11.0,8.0,12.0,8.0,11.0,-7.841667,-4.9,-10.8,5.9,7,7,7,2,5,5,5,0.0,41.666667,66.0,7,5,5,9.1,1.0,3,11.11,0.0,0.327463,0.348212,0.324202,0.317659,0.347122,0.328466,0.33

,AMT
0,4371000
1,2099000
2,3262000
3,6955000
4,6672000
...,...
37363,4573000
37364,10157000
37365,15282000
37366,2328000


In [6]:
def create_lgb_dataset(ds_input, ds_label):
    categorical_features = [ds_input.columns.tolist().index(x) for x in ds_input.columns if ds_input[x].dtype == 'int64']
    
    X_train, X_test, y_train, y_test = train_test_split(ds_input, ds_label, shuffle=True, test_size=0.2, random_state = 42)

    train_data = lgb.Dataset(X_train, label=y_train, categorical_feature=categorical_features,free_raw_data=False)
    test_data = lgb.Dataset(X_test, label=y_test, categorical_feature=categorical_features, free_raw_data=False)

    return train_data, test_data

In [7]:
def mean_absolute_percentage_error(y_true, y_pred):
    return np.mean(np.abs((y_true - y_pred) / y_true)) * 100

In [8]:
def train_and_validate(data, **kwargs):

    num_leaves = kwargs.pop('num_leaves', 31)
    learning_rate = kwargs.pop('learning_rate', 0.1)
    feature_extraction = kwargs.pop('feature_extraction', 1)
    bagging_fraction = kwargs.pop('bagging_fraction', 1)
    max_depth = kwargs.pop('max_depth', -1)
    lambda_l1 = kwargs.pop('lambda_l1', 0.0)
    lambda_l2 = kwargs.pop('lambda_l2', 0.0)
    cat_l2 = kwargs.pop('cat_l2', 10.0)
    cat_smooth = kwargs.pop('cat_smooth', 10.0)
    min_child_weight = kwargs.pop('min_child_weight', 0.001)

    params={'num_leaves':int(num_leaves),
            'learning_rate': learning_rate,
            'feature_extraction': feature_extraction,
            'bagging_fraction': bagging_fraction,
            'max_depth': max_depth,
            'lambda_l1': lambda_l1,
            'lambda_l2': lambda_l2,
            'cat_l2': cat_l2,
            'cat_smooth': cat_smooth,
            'min_child_weight': min_child_weight}

    cv_result = lgb.cv(train_set=data,
                       num_boost_round=1000,
                    #    folds=nfolds, 
                       metrics=['mape'],
                       early_stopping_rounds=50,
                       verbose_eval=200,
                       show_stdv=True,
                       seed=42,
                       params=params)
    

    print("CV MAPE: ", cv_result['mape-mean'])
    
    return max(-np.array(cv_result['mape-mean']))

In [ ]:
def test_best_model(train_data, test_data, bo_optimizer):

    X_train = train_data.construct().get_data()
    y_train = train_data.construct().get_label()
    X_test = test_data.construct().get_data()
    y_test = test_data.construct().get_label()

    params = bo_optimizer.max['params']

    num_leaves = params.pop('num_leaves', 31)
    learning_rate = params.pop('learning_rate', 0.1)
    feature_extraction = params.pop('feature_extraction', 1)
    bagging_fraction = params.pop('bagging_fraction', 1)
    max_depth = params.pop('max_depth', -1)
    lambda_l1 = params.pop('lambda_l1', 0.0)
    lambda_l2 = params.pop('lambda_l2', 0.0)
    cat_l2 = params.pop('cat_l2', 10.0)
    cat_smooth = params.pop('cat_smooth', 10.0)
    min_child_weight = params.pop('min_child_weight', 0.001)

    params={'num_leaves':int(num_leaves),
            'learning_rate': learning_rate,
            'feature_extraction': feature_extraction,
            'bagging_fraction': bagging_fraction,
            'max_depth': max_depth,
            'lambda_l1': lambda_l1,
            'lambda_l2': lambda_l2,
            'cat_l2': cat_l2,
            'cat_smooth': cat_smooth,
            'min_child_weight': min_child_weight}

    params['objective']='regression'
    params['seed'] = 42
    model = lgb.train(train_set=train_ds,
                      valid_sets=test_ds,
                      num_boost_round=10000,
                      early_stopping_rounds=100,
                      verbose_eval=200,
                      params=params)
    
    # 변수 중요도
    lgb.plot_importance(model, figsize=(24, 24))
    plt.show()

    # 예측
    y_pred = model.predict(X_test)
    score = mean_absolute_percentage_error(y_test.reshape(-1, 1), y_pred.reshape(-1, 1))
    print("Test Set MAPE: ", score)

    # 예측 결과 확인
    final_result = pd.concat([pd.Series(y_test, name='label'), pd.Series(y_pred, name='predict')], axis=1)
    sns.regplot(x='label', y='predict', data=final_result)
    plt.show()
    
    return model, score

In [9]:
# LGBM 데이터셋
train_ds, test_ds = create_lgb_dataset(X_data, y_data)
del X_data
del y_data
del data
del df

In [10]:
# 함수 내 변수 고정
train_and_validate_partial = partial(train_and_validate, train_ds)

In [ ]:
# 베이지안 optimization
pbounds = {'num_leaves': (64, 256),
           'learning_rate': (0.001, 0.1),
           'feature_extraction': (0.7, 1.0),
           'bagging_fraction': (0.7, 1.0),
           'lambda_l1': (0, 5),
           'lambda_l2': (0, 5),
           'cat_smooth': (5, 15)}

optimizer = BayesianOptimization(
    f = train_and_validate_partial,
    pbounds = pbounds,
    verbose=2,
    random_state=42
)

optimizer.maximize(init_points=20, n_iter=10)
optimizer.max

|   iter    |  target   | baggin... | cat_sm... | featur... | lambda_l1 | lambda_l2 | learni... | num_le... |
-------------------------------------------------------------------------------------------------------------
[200]	cv_agg's mape: 7.01223 + 0.277732
[400]	cv_agg's mape: 5.57251 + 0.243663
[600]	cv_agg's mape: 5.25908 + 0.212922
[800]	cv_agg's mape: 5.14596 + 0.2092
[1000]	cv_agg's mape: 5.08673 + 0.197346
CV MAPE:  [24.5731458686574, 24.30566097687753, 24.041202222560642, 23.780097120070053, 23.521902901818642, 23.269951411623474, 23.022455187808667, 22.779385132861158, 22.53769750466309, 22.301155867464928, 22.06860630748199, 21.84064164477916, 21.615599623759636, 21.392581206087605, 21.17435133528489, 20.957293819638174, 20.744602157048615, 20.535896662814324, 20.32866049897268, 20.123727337704352, 19.92286053648688, 19.725536760480267, 19.53074994535679, 19.33762319663271, 19.149359474969174, 18.96239487711133, 18.779497753395056, 18.59608026616997, 18.41860817976299, 18.2

In [ ]:
test_best_model(train_ds, test_ds, optimizer)